In [111]:
import json
import pandas as pd
import clasificador as clasificador

In [112]:
with open('large_eve.json', 'r') as file:
    data = [json.loads(line) for line in file]

print(f"Cantidad total de registros: {len(data)}")

Cantidad total de registros: 746909


In [113]:
dns_records = [record for record in data if record.get('event_type') == 'dns']
print(f"Cantidad de registros DNS: {len(dns_records)}")

Cantidad de registros DNS: 15749


In [114]:
print(json.dumps(dns_records[0], indent=2))
print(json.dumps(dns_records[1], indent=2))

{
  "timestamp": "2017-07-22T17:33:16.661646-0500",
  "flow_id": 1327836194150542,
  "pcap_cnt": 22269,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 59680,
  "dest_ip": "2001:0500:0001:0000:0000:0000:803f:0235",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 15529,
    "rrname": "api.wunderground.com",
    "rrtype": "A",
    "tx_id": 0
  }
}
{
  "timestamp": "2017-07-22T17:33:24.990320-0500",
  "flow_id": 2022925111925872,
  "pcap_cnt": 54352,
  "event_type": "dns",
  "vlan": 110,
  "src_ip": "2001:0dbb:0c18:0011:0260:6eff:fe30:0863",
  "src_port": 38051,
  "dest_ip": "2001:0500:0003:0000:0000:0000:0000:0042",
  "dest_port": 53,
  "proto": "UDP",
  "dns": {
    "type": "query",
    "id": 58278,
    "rrname": "stork79.dropbox.com",
    "rrtype": "A",
    "tx_id": 0
  }
}


In [115]:
df = pd.json_normalize(dns_records)
print(f"Shape del dataframe: {df.shape}")

Shape del dataframe: (15749, 18)


In [116]:
df_type_a = df[df['dns.rrtype'] == 'A']
print(f"Cantidad de registros tipo A: {len(df_type_a)}")

Cantidad de registros tipo A: 2849


In [117]:
unique_domains = df_type_a['dns.rrname'].unique()
print(f"Cantidad de dominios únicos: {len(unique_domains)}")

Cantidad de dominios únicos: 177


In [118]:
def get_TLD(domain, ngram):
    parts = domain.split('.')
    ngram_parts = ngram.split('.')
    if len(ngram_parts) > 0:
        for i in range(len(parts) - 1, -1, -1):
            if parts[i] in ngram_parts:
                return '.'.join(parts[i-1:])
    return domain

# Example usage
domain1 = "api.wunderground.com"
domain2 = "safebrowsing.clients.google.com.home"

ngram1 = "com.wunderground.api"  # Ngrams for domain1
ngram2 = "home.com.google.clients.safebrowsing"  # Ngrams for domain2

print("TLD for", domain1, ":", get_TLD(domain1, ngram1))
print("TLD for", domain2, ":", get_TLD(domain2, ngram2))




TLD for api.wunderground.com : wunderground.com
TLD for safebrowsing.clients.google.com.home : com.home


In [119]:
df_unique_domains = pd.DataFrame({'domain': unique_domains})
df_unique_domains['domain_tld'] = df_unique_domains['domain'].apply(get_tld)

#columns_to_drop = ['length', 'digits', 'entropy', 'vowel-cons', 'firstDigitIndex', 'ngrams']

#df_unique_domains = df_unique_domains.drop(columns=columns_to_drop)
print(f"Shape of the dataframe with TLDs: {df_unique_domains.shape}")

Shape of the dataframe with TLDs: (177, 2)


In [120]:
#ahora vamos con la parte 2

In [121]:
df_unique_domains

,domain,domain_tld
0,api.wunderground.com,com
1,stork79.dropbox.com,com
2,hpca-tier2.office.aol.com.ad.aol.aoltw.net,net
3,safebrowsing.clients.google.com.home,com.home
4,fxfeeds.mozilla.com,com
...,...,...
172,ntp.ubuntu.com.localdomain,com.localdomain
173,redir.metaservices.microsoft.com,com
174,ocsp.verisign.com,com
175,192.168.22.201:.stayonline.net,net


In [99]:
df_classified = clasificador.clasificacion(df_unique_domains)

ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]